In [ ]:
pip install -U langsmith

In [50]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
loader = TextLoader('source.txt')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,)

split_documents = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

chroma_document_store = Chroma.from_documents(split_documents, embeddings)

print("There are", chroma_document_store._collection.count(), "in the collection")

retriever = chroma_document_store.as_retriever(search_kwargs={"k": 4})


There are 71 in the collection


In [157]:

model = ChatOpenAI(model_name="gpt-4", temperature=0)

# Define prompt template
template = """You are a helpful documentation Q&A assistant, trained to answer questions about ultra marathon running. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use two sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline





rag_chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)

rag_chain.invoke({"question": "will eating beetroot make me run further"})





'The documents provided do not contain specific information about the effects of eating beetroot on running performance or endurance.'

In [126]:
import os
from langsmith import Client
from langsmith.evaluation import LangChainStringEvaluator, evaluate
from langsmith.schemas import Run, Example

os.environ["LANGCHAIN_API_KEY"] = ""
os.environ['OPENAI_API_KEY'] = ''
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_PROJECT'] = "book-chapter-9" 


# Initialize the LangSmith client
client = Client()

In [136]:
# Define the dataset
dataset_name = "Ultra Marathon QA Dataset"
description = "Dataset for evaluating QA correctness of an ultra marathon chatbot."

# Create the dataset in LangSmith
dataset = client.create_dataset(dataset_name, description=description)

# Add examples to the dataset
client.create_examples(
    inputs=[
        {"question": "How do you train for an ultra marathon?"},
        {"question": "will i need to buy different sized trainers?"}
    ],
    outputs=[
        {"answer": "Training involves increasing mileage gradually with long runs and recovery periods."},
        {"answer": "Consider buying trainers that are a half-size larger to accommodate swelling feet during long runs."}
    ],
    dataset_id=dataset.id,
)


In [156]:
def predict(inputs: dict):
    return rag_chain.invoke({"question": inputs["question"]})


def format_evaluator_inputs(run: Run, example: Example):
    return {
        "input": example.inputs["question"],
        "prediction": next(iter(run.outputs.values())),
        "reference": example.outputs["answer"],
    }


correctness_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={"criteria": "correctness", "normalize_by": 10},
    prepare_data=format_evaluator_inputs,
)

results = evaluate(
    predict,
    data=dataset_name,
    experiment_prefix="Chat Single Turn",
    evaluators=[correctness_evaluator],
    metadata={"model": "gpt-3.5-turbo"},
)

View the evaluation results for experiment: 'Chat Single Turn-8f72e190' at:
https://smith.langchain.com/o/d0accd60-ea2b-5fb1-8194-a9f975563f7e/datasets/0eb55f05-964c-444e-b69b-d85b5ed02faa/compare?selectedSessions=fe12e2ae-43b8-409a-9891-8a8dce6e289d




3it [00:18,  6.28s/it]


In [116]:
results = evaluate(
    chatbot_task,            # Your chatbot task function
    dataset_name,            # The dataset to evaluate on
    [correctness_evaluator]  # The evaluator function
)

# import langsmith
# from langchain import chat_models, prompts, smith
# from langchain.schema import output_parser



# # Define the evaluators to apply
# eval_config = smith.RunEvalConfig(
#     evaluators=[
#         "qa"
#     ],
#     eval_llm=chat_models.ChatOpenAI(model="gpt-4", temperature=0)
# )


# chain_results = client.run_on_dataset(
#     dataset_name="Ultra Marathon QA Dataset",
#     llm_or_chain_factory=rag_chain,
#     evaluation=eval_config,
#     project_name="run007",
#     concurrency_level=5,
#     verbose=True,
# )

View the evaluation results for project 'run007' at:
https://smith.langchain.com/o/d0accd60-ea2b-5fb1-8194-a9f975563f7e/datasets/d1a15a3e-0566-4268-85f0-a261ff4f426b/compare?selectedSessions=4bdb80f7-40b4-4fe5-a7d0-511222237784

View all tests for Dataset Ultra Marathon QA Dataset at:
https://smith.langchain.com/o/d0accd60-ea2b-5fb1-8194-a9f975563f7e/datasets/d1a15a3e-0566-4268-85f0-a261ff4f426b


Chain failed for example 5d2532f7-3b56-4392-9028-f6cfcc5c58ee with inputs {'input': 'will i need to buy different sized running shoes'}
Error Type: KeyError, Message: 'question'


[------------------------------------------------->] 1/1

,error,execution_time,run_id
count,1,1.000000,1
unique,1,NaN,1
top,'question',NaN,b2a683a2-0b62-4950-903c-e2003d1851d3
freq,1,NaN,1
mean,NaN,0.063151,NaN
std,NaN,NaN,NaN
min,NaN,0.063151,NaN
25%,NaN,0.063151,NaN
50%,NaN,0.063151,NaN
75%,NaN,0.063151,NaN
